In [5]:
# Bibliotecas necessárias
!pip install -U sentence-transformers faiss-cpu pandas openai

In [ ]:
import pandas as pd

# Caminho JSON lOCAL
caminho_json = r'C:\Users\Willgnner\Documents\TJ-GO\Conciliacao\Dados\dados_processos_cejusc_14052025.json'

# Carrega os dados
df = pd.read_json(caminho_json, lines=True)
df = df.dropna(subset=['inteiro_teor'])  # Remove registros sem texto
df.reset_index(drop=True, inplace=True)

print(f"{len(df)} documentos carregados.")
df[['numero_processo', 'classificacao']].head()

16583 documentos carregados.


,numero_processo,classificacao
0,5437274.43.2024.8.09.0012,frutifero
1,5437284.21.2024.8.09.0131,infrutifero
2,5438477.44.2024.8.09.0140,frutifero
3,5437321.31.2024.8.09.0169,infrutifero
4,5438510.45.2024.8.09.0007,infrutifero


In [7]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# Carrega o modelo
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Monta corpus com dados enriquecidos
corpus = []
for idx, row in df.iterrows():
    combinado = f"""
    [INTEIRO_TEOR]
    {row['inteiro_teor']}

    [Polo Passivo]: {row.get('polo_passivo', '')}
    [Tipo Parte Passiva]: {row.get('cpf_cnpj_polo_passivo', '')}
    [Assunto]: {row.get('assunto', '')}
    [Classe]: {row.get('classe', '')}
    """.strip()
    corpus.append(combinado)

# Salva esse texto no próprio DataFrame
df['texto_embedding'] = corpus

# Geração em batch
batch_size = 16
embeddings = []

for i in tqdm(range(0, len(corpus), batch_size), desc="Gerando embeddings"):
    batch = corpus[i:i + batch_size]
    emb = modelo_embedding.encode(batch, show_progress_bar=False)
    embeddings.extend(emb)

embeddings = np.array(embeddings)
print(f"Embeddings gerados: {embeddings.shape}")

c:\Users\Willgnner\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gerando embeddings: 100%|██████████| 1037/1037 [02:17<00:00,  7.54it/s]

Embeddings gerados: (16583, 384)


In [8]:
# Salvar os embeddings
np.save("embeddings_cejusc.npy", embeddings)

# Salvar os dados completos
df.to_csv("dados_processos_completo.csv", index=False, encoding='utf-8')
print("Dados e embeddings salvos.")

Dados e embeddings salvos.


In [9]:
import faiss

# Indexação com FAISS
embedding_dim = embeddings.shape[1]
embeddings_np = embeddings.astype('float32')

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

print(f"Index FAISS criado com {index.ntotal} vetores.")

Index FAISS criado com 16583 vetores.


In [ ]:
import openai

# Cliente Maritaca
client = openai.OpenAI(
    api_key="COLOQUE_SUA_API_KEY_AQUI",
    base_url="https://chat.maritaca.ai/api"
)

def classificar_peticao(texto, k=10):
    # Gera embedding do novo texto
    emb_novo = modelo_embedding.encode([texto])[0].astype('float32')

    # Recupera documentos similares no índice FAISS
    _, indices = index.search(np.array([emb_novo]), k)
    docs_similares = df.iloc[indices[0]]['inteiro_teor'].tolist()
    contexto = "\n\n---\n\n".join(docs_similares)

    # Prompt completo da área de negócio
    prompt = f"""
Você é um assistente jurídico treinado para avaliar petições judiciais com base no seu conteúdo completo.
Sua tarefa é classificar o processo da seguinte forma:
    • Classificação: frutífero ou infrutífero
    • Justificativa: Explique de forma direta e fundamentada por que a ação tem ou não chance de realização de acordo (conciliação), com base nos critérios abaixo.

Avalie com base nos seguintes critérios (de forma técnica, objetiva e crítica):
    1. Parte ré (polo passivo):
        ◦ Se for uma empresa privada (CNPJ) e não houver provas documentais relevantes (como nota fiscal, contrato, prints, laudos, e-mails ou mensagens), classifique como infrutífero.
        ◦ Se for órgão público e o pedido envolver direitos sociais essenciais (como benefícios, saúde ou sustento), e houver documentos mínimos, pode ser frutífero. 
    2. Parte autora (polo ativo):
        ◦ Pessoa física em condição de hipossuficiência evidente (como trabalhador rural, doente, idoso, pessoa de baixa renda) aumenta a chance de êxito — classifique como frutífero se o pedido for coerente e verossímil. 
    3. Provas juntadas ou mencionadas:
        ◦ Ausência total de provas → infrutífero.
        ◦ Prints de conversa, contratos, comprovantes de pagamento, relatórios, prints de sistemas, laudos técnicos ou e-mails → aumentam a viabilidade do acordo.
    4. Urgência ou perigo da demora:
        ◦ Risco imediato à subsistência, moradia, renda, tratamento médico ou integridade física justifica uma avaliação positiva, mesmo com documentação limitada → frutífero.
    5. Valor da causa:
        ◦ Valor baixo com documentação consistente = frutífero.
        ◦ Valor baixo com pedido genérico ou sem provas = infrutífero.
        ◦ Valor alto, sem elementos probatórios suficientes, não altera a classificação → continua infrutífero.
    6. Clareza e estrutura do pedido:
        ◦ Petições com narrativa confusa, sem delimitação de pedidos ou argumentos dispersos → infrutífero.
        ◦ Petições com exposição clara dos fatos, pedidos objetivos e fundamentação direta → frutífero.
    7. Fundamentação jurídica e jurisprudência:
        ◦ Referências a jurisprudência pertinente, súmulas ou precedentes vinculantes reforçam a credibilidade do pedido → ponto positivo.
    8. Potencial de improcedência:
        ◦ Se a procedência depende de prova não anexada, inexistente ou de produção improvável, classifique como infrutífero.

Formato da resposta (obrigatório):
Classificação: frutífero ou infrutífero
Justificativa: [máximo 3 frases, linguagem direta e técnica]

Texto da nova petição:
{texto}

Outros documentos similares:
{contexto}

Resposta:
"""

    # Chamada à API Maritaca
    resposta = client.chat.completions.create(
        model="sabia-3.1",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000
    )

    return resposta.choices[0].message.content.strip()

In [11]:
# Testar com um documento da própria base (ex: índice 125)
texto_teste = df.loc[21, 'inteiro_teor']
resposta = classificar_peticao(texto_teste)
print(resposta)

Classificação: frutífero

Justificativa: A parte autora é trabalhadora rural, reconhecidamente hipossuficiente e portadora de várias patologias graves que a incapacitam permanentemente para o trabalho, conforme atestado por profissionais médicos especialistas. A documentação médica apresentada e os laudos comprovam a incapacidade laborativa, justificando o pedido de restabelecimento de auxílio-doença rural e a concessão de aposentadoria por invalidez rural com adicional de 25%. Além disso, a urgência da medida é evidente devido ao caráter alimentar do benefício, essencial para a subsistência da autora e de sua família. A autora também demonstrou ter sido segurada especial, comprovando sua condição de trabalhadora rural sob regime de economia familiar, o que reforça a verossimilhança de suas alegações e justifica a concessão da tutela antecipada.


In [ ]:
import re
import unicodedata

def normalizar_texto(texto):
    texto = texto.lower().strip()
    return unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8')

def extrair_classificacao(resposta):
    resposta_norm = normalizar_texto(resposta)

    # Busca padrão "Classificação: frutifero" (com ou sem acento, com ou sem espaços)
    match = re.search(r'classificacao:\s*(frutifero|infrutifero)', resposta_norm)
    if match:
        return match.group(1)

    # Fallback por palavra solta
    if "infrutifero" in resposta_norm:
        return "infrutifero"
    elif "frutifero" in resposta_norm:
        return "frutifero"
    
    return "indefinido"

In [14]:
from tqdm import tqdm
import pandas as pd

def avaliar_llm_tempo_real(df, k=20, limite=100, seed=2024):
    resultados = []
    total = len(df) if limite is None else min(limite, len(df))
    acertos = 0

    # Amostragem aleatória com seed variável
    amostra_indices = df.sample(n=total, random_state=seed).index.tolist()

    for i, idx in enumerate(tqdm(amostra_indices, desc=f"Avaliando LLM (seed={seed})")):
        linha = df.loc[idx]
        texto = linha['texto_embedding']
        real = normalizar_texto(linha['classificacao'])

        try:
            resposta = classificar_peticao(texto, k=k)
            previsto = extrair_classificacao(resposta)
            match = previsto == real
            if match:
                acertos += 1

            resultados.append({
                "indice": idx,
                "real": real,
                "previsto": previsto,
                "match": match,
                "resposta_bruta": resposta
            })

            print(f"\n---\nResposta bruta da LLM:\n{resposta}\n")
            print(f"[{i+1}/{total}] Real: {real} | Previsto: {previsto} | Match: {match} | Acurácia parcial: {acertos / (i+1):.2%}")

        except Exception as e:
            resultados.append({
                "indice": idx,
                "real": real,
                "previsto": "erro",
                "match": False,
                "resposta_bruta": str(e)
            })
            print(f"[{i+1}/{total}] ERRO: {e} | Acurácia parcial: {acertos / (i+1):.2%}")

    df_resultado = pd.DataFrame(resultados)
    print(f"\nAcurácia final (seed {seed}): {df_resultado['match'].mean():.2%}")
    return df_resultado

In [16]:
seeds = [42, 2023, 99, 7, 123]
resultados_acuracia = []

for seed in seeds:
    print(f"\n\nAvaliando com seed {seed}...\n")
    df_result = avaliar_llm_tempo_real(df, k=10, limite=100, seed=seed)
    
    acuracia = df_result['match'].mean()
    resultados_acuracia.append({
        "seed": seed,
        "acuracia": round(acuracia * 100, 2)
    })



Avaliando com seed 42...



Avaliando LLM (seed=42):   1%|          | 1/100 [00:08<14:03,  8.52s/it]


---
Resposta bruta da LLM:
Classificação: frutífero

Justificativa: A parte ré é uma pessoa física e não um órgão público, e a petição apresenta provas documentais claras, como boletos e contratos, que evidenciam o débito e a prestação de serviços. Além disso, o valor da causa é baixo, mas com documentação consistente, o que indica uma chance razoável de acordo em conciliação. A narrativa dos fatos é clara e os pedidos são objetivos e fundamentados, reforçando a viabilidade de uma solução conciliatória.

[1/100] Real: infrutifero | Previsto: frutifero | Match: False | Acurácia parcial: 0.00%


Avaliando LLM (seed=42):   2%|▏         | 2/100 [00:14<11:53,  7.28s/it]


---
Resposta bruta da LLM:
Classificação: infrutífero
Justificativa: A petição trata de uma ação de cobrança em um juizado especial cível, onde o requerido é uma pessoa física e não há indicação de provas documentais relevantes como contratos, notas fiscais ou comprovantes de pagamento. Além disso, os pedidos são genéricos em relação às outras petições similares apresentadas, sem especificação clara de provas ou valores, o que reduz a viabilidade de um acordo.

[2/100] Real: frutifero | Previsto: infrutifero | Match: False | Acurácia parcial: 0.00%


...


---
Resposta bruta da LLM:
Classificação: Frutífero
Justificativa: A parte autora, Luciana Ferreira Duarte, é pessoa física hipossuficiente e apresentou provas documentais robustas, como cheques sem fundo e relatório de ordem de serviços, que comprovam a prestação dos serviços e a inadimplência da parte ré, Maycon Frenhan Luz. A urgência é evidenciada pelo risco de prejuízos financeiros contínuos e a possibilidade de negociação está apoiada na existência de tais documentos. Além disso, o valor da causa é significativo, mas a documentação apresentada é consistente, o que reforça a viabilidade de um acordo.

[98/100] Real: frutifero | Previsto: frutifero | Match: True | Acurácia parcial: 50.00%


Avaliando LLM (seed=123):  99%|█████████▉| 99/100 [48:42<00:15, 15.37s/it]


---
Resposta bruta da LLM:
Classificação: frutífero
Justificativa: A petição é frutífera pois apresenta provas documentais relevantes, como notas fiscais, comprovantes de entrega de mercadorias e duplicatas protestadas, que demonstram a prestação de serviços e o inadimplemento da parte ré. Ademais, o valor da causa não é considerado baixo, mas a documentação é consistente e há clara fundamentação jurídica com base na Lei 5.474/68 e no Código Civil, reforçando a viabilidade do acordo. A urgência não é explicitamente mencionada, mas a necessidade de recuperação do crédito e a execução das duplicatas justificam a ação de execução.

[99/100] Real: infrutifero | Previsto: frutifero | Match: False | Acurácia parcial: 49.49%


Avaliando LLM (seed=123): 100%|██████████| 100/100 [50:41<00:00, 30.41s/it]


---
Resposta bruta da LLM:
Classificação: frutífero
Justificativa: A parte ré, Ana Cristina Dourado de Sousa, é uma pessoa física, e o pedido envolve a prestação de serviços de acesso à internet, que é um direito essencial e relacionado a serviços sociais. A parte autora, RADAR WISP LTDA – ME, é uma empresa de pequeno porte (EPP) com um faturamento bruto de R$3.286.073,19 no período de 12 meses, o que indica uma situação financeira estável e capacidade para cumprir acordos. Além disso, há provas documentais, como contratos de prestação de serviços, notas fiscais e comprovantes de inscrição no CNPJ, que sustentam a legitimidade da cobrança e a existência de um vínculo contratual entre as partes. A urgência do serviço de acesso à internet para fins essenciais e a clareza na descrição do objeto e das obrigações contratuais também favorecem a possibilidade de conciliação.

[100/100] Real: infrutifero | Previsto: frutifero | Match: False | Acurácia parcial: 49.00%

Acurácia final (seed 123

In [17]:
df_avaliacoes = pd.DataFrame(resultados_acuracia)
print("\nAcurácia por seed:\n")
print(df_avaliacoes)


Acurácia por seed:

   seed  acuracia
0    42      52.0
1  2023      45.0
2    99      48.0
3     7      49.0
4   123      49.0
